In [ ]:
import sys
  
# printing all directories for 
# interpreter to search

sys.path.append('/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit')
sys.path

['/content',
 '/env/python',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.8/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit']

In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.base import clone
import shlex
import csv
import re
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import gensim
from gensim.models import KeyedVectors
from gensim import models
from sklearn.decomposition import PCA
import pickle
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from urllib.parse import urlparse
import helper

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Creating training dataframe

In [ ]:
%%time
r = requests.get('https://demo.osapiens.cloud/data/integration/supplier-os-hub/supplier-os-data-provider/ExternalNewsExport/?type=LABELED&limit=100000&username=pujit.golecha@gmail.com&password=Golchha@1999')

CPU times: user 377 ms, sys: 74.5 ms, total: 452 ms
Wall time: 1min 5s


In [ ]:
r.text

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#new end point has no headers
header=["id","ts","lang","source","title","content","confirmedLabel"]
header

['id', 'ts', 'lang', 'source', 'title', 'content', 'confirmedLabel']

In [ ]:
raw_string=r.text.replace('\r\n"','"')
raw_string=re.sub(r'id,ts,source,title,content,confirmedLabel\r\n','',r.text)
raw_string=re.sub(r'([^\d])(\r\n)+', r'\1',raw_string)


In [ ]:
raw_string

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
data=[]
for rowno,i in enumerate(tqdm(raw_string.split("\r\n"))):
  text=''
  a=[]
  #z is indicator variable if the string contains ""
  z=False
  for j in i.split(","):
    #some columns may be empty
    if j!="":
      #accounting for ""
      #we continue concatenating all parts to the text until we encounter another"
      if j[0]=='"' or z==True:
        text=text+j
        z=True
      else:
        a.append(j)
      try:
        if j[-1]=='"' and j[-2]!='"':
          a.append(text)
          text=''
          z=False
      #handling the case when comma occurs before closed quotes and then there is another comma which is the separator eg "Hi,",
      except:
         if j[-1]=='"':
          print(j)
          a.append(text)
          print(rowno)
          print(text)
          text=''
          z=False
    else:
      a.append("")


  data.append(a)

  0%|          | 0/54485 [00:00<?, ?it/s]

"
105
"Jennifer Nichols meets an apiarist who has mastered the skill of rescuing bees from inside homes and buildings"
"
2644
"A new FBI report show a 50% jump in active-shooter incidents from 2020 to 2021"
"
10877
"President Joe Biden has tested positive for Covid-19 and is experiencing ""mild symptoms"" White House press secretary Karine Jean-Pierre said Thursday."
"
12581
"The reasons of those who identify Pujol as the cause and origin of the Catalan decline are powerful both for his responsibilities regarding the independence process and corruption"


 82%|████████▏ | 44935/54485 [00:00<00:00, 113397.58it/s]

"
14681
"Leclerc wins by a landslide and Sainz gets burned in the Austrian Grand Prix"
"
28238
"The 75th edition of the prestigious Cannes Film Festival witnesses the return of international auteurs Palme d'Or winning filmmakers potential summer blockbusters and many films that will"
"
28405
"Rohit who was playing in India's ongoing four-day warm-up game against Leicestershire ahead of the rescheduled fifth Test against England has gone into quarantine. His availability for the Test beginning July 1 in Edgbaston will now be subject to his recovery"
"
32289
"Compared to its ancestors the latest Omicron subvariant BA.5 may have an enhanced ability to create a large number of copies of the coronavirus once it gets into human cells"
"
34324
"Tom 29 works hard as a cattle rancher but he has always dreamed of driving a DeLorean. The world-famous car featured in the Back to the Future trilogy"
"
35813
"Published in 1984 the famous novel by Marguerite Duras was awarded the Goncourt Prize the s

100%|██████████| 54485/54485 [00:00<00:00, 104464.22it/s]


In [ ]:
data=pd.DataFrame(data, columns=header)

In [ ]:
data.drop(len(data)-1,axis=0,inplace=True)

In [ ]:
data

,id,ts,lang,source,title,content,confirmedLabel
0,abc.es-0f16d41b22631657500284f423cab35af5767edf,1665082362000,es,abc.es,Three seriously injured in the explosion of so...,"""Several explosions in the commercial premises...",2
1,abc.es-0fe9db0c206d6f613fb270a3557d4cfd2732b05b,1666856250000,es,abc.es,A seven-meter python attacks and engulfs a wom...,"""The body of a woman has been found in the sto...",0
2,abc.es-10d56ae3e9d99c905ede9ccf4c4b09f7c353acc9,1666453249000,es,abc.es,Worker dies in a construction site at Repsol's...,"""An employee of an auxiliary company working o...",2
3,abc.es-14c8a574d028dd3e7091cb270867993e19f56095,1665850543000,es,abc.es,International community turns to Turkey in the...,"""Forty-one people were killed and 28 injured i...",2
4,abc.es-17477e7fd1f064e263633dba8bb2eb39d538cbf2,1667476276000,es,abc.es,Air conditioning problems reported in the reha...,"""UGT Public Services has denounced before the ...",2
...,...,...,...,...,...,...,...
54479,yahoo.com-ff86305c30e24585534e1064dd52483aed71...,1653150583000,en-US,yahoo.com,A Maine town reportedly shut down after its on...,,0
54480,yahoo.com-ffb99a82ff176686aa6cd70b5de3134418c0...,1652871195000,en-US,yahoo.com,"""High-profile candidates endorsed by Trump los...",,0
54481,yahoo.com-ffcc76f8ebce23fee39d5df0965ec86924b6...,1653477680000,en-US,yahoo.com,China Protesters Demand Back Billions Invested...,,0
54482,yahoo.com-ffce6e50376bb174010c8b9c878826d13735...,1652896045000,en-US,yahoo.com,Judge: Trump administration illegally withdrew...,,0


In [ ]:
data=data[~((data["title"].isna()) & (data["content"].isna()))]
data=data[~data["confirmedLabel"].isna()]

In [ ]:
data.loc[data["confirmedLabel"]=="2"]

,id,ts,lang,source,title,content,confirmedLabel
0,abc.es-0f16d41b22631657500284f423cab35af5767edf,1665082362000,es,abc.es,Three seriously injured in the explosion of so...,"""Several explosions in the commercial premises...",2
2,abc.es-10d56ae3e9d99c905ede9ccf4c4b09f7c353acc9,1666453249000,es,abc.es,Worker dies in a construction site at Repsol's...,"""An employee of an auxiliary company working o...",2
3,abc.es-14c8a574d028dd3e7091cb270867993e19f56095,1665850543000,es,abc.es,International community turns to Turkey in the...,"""Forty-one people were killed and 28 injured i...",2
4,abc.es-17477e7fd1f064e263633dba8bb2eb39d538cbf2,1667476276000,es,abc.es,Air conditioning problems reported in the reha...,"""UGT Public Services has denounced before the ...",2
6,abc.es-2917c8e0c285bdb69f56bfa639d68a0fec09230e,1665246489000,es,abc.es,A worker of a construction company falls from ...,"""A 53-year-old worker was injured Friday after...",2
...,...,...,...,...,...,...,...
53724,yahoo.com-7fd6f278080104fb4ac0a4fb5b7e895849fb...,1654201614000,en-US,yahoo.com,"""Coal pile slide buries kills 2 at Colorado po...",,2
54142,yahoo.com-c467c80cf65b27f3d8940b418161af366e43...,1654259862000,en-US,yahoo.com,"""Popeyes employee recorded hitting a teen work...",,2
54245,yahoo.com-d365897cd98bd1553d8422c5c9b59aa463f9...,1655241097000,en-US,yahoo.com,"""A merchant navy cadet is suing Maersk saying ...",,2
54442,yahoo.com-f8e96ec3d8d54b58a21224063dfddb407dec...,1657377333000,en-US,yahoo.com,Suncor Replaces CEO Mark Little After Oil Sand...,,2


In [ ]:
data.to_csv("/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit/new_training_data.csv",index=False)

# Embeddings creation for train set

In [ ]:
w2v_model = models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
data= pd.read_csv("/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit/new_training_data.csv")
data

In [ ]:
data["confirmedLabel"].value_counts()

0    52097
2     1262
1      116
Name: confirmedLabel, dtype: int64

In [ ]:
[helper.preprocessing(title) for title in tqdm(list(data["title"][0:10]))]

100%|██████████| 10/10 [00:02<00:00,  4.57it/s]


[['three',
  'seriously',
  'injured',
  'explosion',
  'commercial',
  'premise',
  'alcorcón'],
 ['seven',
  'meter',
  'python',
  'attack',
  'engulfs',
  'woman',
  'wa',
  'working',
  'rubber',
  'plantation'],
 ['worker', 'dy', 'construction', 'site', 'repsol', 'coruña', 'refinery'],
 ['international',
  'community',
  'turn',
  'turkey',
  'aftermath',
  'tragic',
  'coal',
  'mine',
  'accident'],
 ['air',
  'conditioning',
  'problem',
  'reported',
  'rehabilitation',
  'service',
  'new',
  'toledo',
  'hospital'],
 ['mauricio',
  'claver',
  'carone',
  'former',
  'idb',
  'president',
  'wa',
  'threatened',
  'extorted',
  'president',
  'argentina'],
 ['worker',
  'construction',
  'company',
  'fall',
  'height',
  'six',
  'meter',
  'sigüenza'],
 ['compensation',
  'claim',
  'eye',
  'injury',
  'caused',
  'bull',
  'fire',
  'seville',
  'upheld'],
 ['military',
  'sink',
  'boat',
  'cuban',
  'migrant',
  'fleeing',
  'island',
  'causing',
  'seven',
  'death

In [ ]:
corpus_title = [helper.preprocessing(title) for title in tqdm(list(data["title"]))]      

100%|██████████| 53475/53475 [00:21<00:00, 2523.38it/s]


In [ ]:
corpus = [helper.preprocessing(title) for title in tqdm(list(data["content"]))]

100%|██████████| 53475/53475 [00:38<00:00, 1384.90it/s]


In [ ]:
corpus

In [ ]:
finaldf=[]
for i,j,k,l in tqdm(zip(corpus,data["id"],data["confirmedLabel"],corpus_title)):
  try:
    doc_Embedding=helper.document_vector(i)
    finaldf.append({"id":j,"label":k,"emebedding":doc_Embedding})
  except:
    try:
      doc_Embedding=helper.document_vector(l)
      finaldf.append({"id":j,"label":k,"emebedding":doc_Embedding})
    except:
      pass


53475it [00:08, 6184.68it/s]


In [ ]:
finaldf=pd.DataFrame(finaldf)

In [ ]:
finaldf

,id,label,emebedding
0,abc.es-0f16d41b22631657500284f423cab35af5767edf,2,"[0.02797421, -0.0041298456, 0.027603695, 0.047..."
1,abc.es-0fe9db0c206d6f613fb270a3557d4cfd2732b05b,0,"[0.036916878, 0.029810978, -0.023043118, 0.000..."
2,abc.es-10d56ae3e9d99c905ede9ccf4c4b09f7c353acc9,2,"[-0.0139687415, 0.008222221, 0.04526988, -0.00..."
3,abc.es-14c8a574d028dd3e7091cb270867993e19f56095,2,"[0.011251955, 0.01864549, 0.0417256, 0.0427638..."
4,abc.es-17477e7fd1f064e263633dba8bb2eb39d538cbf2,2,"[-0.028778076, 0.04251526, 0.031568658, 0.0410..."
...,...,...,...
53464,yahoo.com-ff86305c30e24585534e1064dd52483aed71...,0,"[0.041836213, 0.034118652, -0.029907227, -0.08..."
53465,yahoo.com-ffb99a82ff176686aa6cd70b5de3134418c0...,0,"[-0.029369008, 0.02598433, 0.0043945312, 0.028..."
53466,yahoo.com-ffcc76f8ebce23fee39d5df0965ec86924b6...,0,"[0.09580994, 0.05645752, 0.04663849, 0.0687255..."
53467,yahoo.com-ffce6e50376bb174010c8b9c878826d13735...,0,"[-0.018859863, 0.05102539, -0.06697954, -0.030..."


In [ ]:
finaldf[list(range(0, 300))] = pd.DataFrame(finaldf.emebedding.tolist(), index= finaldf.index)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


In [ ]:
finaldf.drop(columns='emebedding', inplace= True)
finaldf['label'] = finaldf['label'].astype(int)
finaldf = finaldf[finaldf['label']!=1]
finaldf['label'] = finaldf['label'].replace(2,1)

<ipython-input-14-347712e8bf73>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finaldf['label'] = finaldf['label'].replace(2,1)


In [ ]:
finaldf

,id,label,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,abc.es-0f16d41b22631657500284f423cab35af5767edf,1,0.027974,-0.004130,0.027604,0.047051,-0.042813,-0.030466,0.031732,-0.109924,...,-0.027727,0.020912,-0.063067,0.010870,-0.007165,-0.008956,-0.055347,-0.076288,-0.034178,0.011439
1,abc.es-0fe9db0c206d6f613fb270a3557d4cfd2732b05b,0,0.036917,0.029811,-0.023043,0.000561,-0.046248,-0.024618,0.033148,-0.089768,...,-0.086267,0.005678,-0.071661,0.043474,-0.004877,-0.036751,-0.038906,-0.008480,0.027132,0.020889
2,abc.es-10d56ae3e9d99c905ede9ccf4c4b09f7c353acc9,1,-0.013969,0.008222,0.045270,-0.001446,-0.036408,-0.027278,0.060472,-0.086428,...,-0.045860,-0.022844,-0.085090,0.039385,-0.009400,0.015813,-0.040487,-0.056268,0.014432,-0.014655
3,abc.es-14c8a574d028dd3e7091cb270867993e19f56095,1,0.011252,0.018645,0.041726,0.042764,-0.007114,-0.032907,0.042889,-0.108137,...,-0.057986,-0.039835,-0.098461,0.028492,-0.035461,-0.019045,-0.041769,-0.018278,0.049495,-0.005421
4,abc.es-17477e7fd1f064e263633dba8bb2eb39d538cbf2,1,-0.028778,0.042515,0.031569,0.041072,-0.059480,0.008428,0.071166,-0.092485,...,-0.055534,0.013167,-0.081876,0.061822,-0.025959,0.020732,-0.036089,-0.035680,0.011537,-0.004418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53464,yahoo.com-ff86305c30e24585534e1064dd52483aed71...,0,0.041836,0.034119,-0.029907,-0.084251,-0.030286,0.035919,-0.020428,-0.095990,...,-0.011285,0.047606,-0.128213,0.102539,0.003723,0.010059,-0.070508,-0.070190,0.011304,0.046594
53465,yahoo.com-ffb99a82ff176686aa6cd70b5de3134418c0...,0,-0.029369,0.025984,0.004395,0.028553,-0.110529,-0.052934,-0.006775,-0.096824,...,0.017234,-0.060626,-0.095481,-0.011397,-0.051758,-0.061146,0.081326,-0.115656,-0.085305,0.033836
53466,yahoo.com-ffcc76f8ebce23fee39d5df0965ec86924b6...,0,0.095810,0.056458,0.046638,0.068726,-0.048553,-0.021271,0.007395,-0.130402,...,-0.039978,0.028349,-0.049713,-0.004752,0.070587,0.080563,-0.049167,-0.061920,0.077354,-0.077606
53467,yahoo.com-ffce6e50376bb174010c8b9c878826d13735...,0,-0.018860,0.051025,-0.066980,-0.030727,-0.037231,-0.177595,-0.013742,0.033779,...,0.012521,0.023455,-0.110944,-0.009347,0.094432,0.031634,0.103768,-0.120919,0.057426,0.046701


In [ ]:
finaldf.to_csv("/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit/new_train_w_2_vec.csv",index=False)

# Performance Evaluation

In [ ]:
finaldf = pd.read_csv('/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit/new_train_w_2_vec.csv')
finaldf

,id,label,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,abc.es-0f16d41b22631657500284f423cab35af5767edf,1,0.027974,-0.004130,0.027604,0.047051,-0.042813,-0.030466,0.031732,-0.109924,...,-0.027727,0.020912,-0.063067,0.010870,-0.007165,-0.008956,-0.055347,-0.076288,-0.034178,0.011439
1,abc.es-0fe9db0c206d6f613fb270a3557d4cfd2732b05b,0,0.036917,0.029811,-0.023043,0.000561,-0.046248,-0.024618,0.033148,-0.089768,...,-0.086267,0.005678,-0.071661,0.043474,-0.004877,-0.036751,-0.038906,-0.008480,0.027132,0.020889
2,abc.es-10d56ae3e9d99c905ede9ccf4c4b09f7c353acc9,1,-0.013969,0.008222,0.045270,-0.001446,-0.036408,-0.027278,0.060472,-0.086428,...,-0.045860,-0.022844,-0.085090,0.039385,-0.009400,0.015813,-0.040487,-0.056268,0.014432,-0.014655
3,abc.es-14c8a574d028dd3e7091cb270867993e19f56095,1,0.011252,0.018645,0.041726,0.042764,-0.007114,-0.032907,0.042889,-0.108137,...,-0.057986,-0.039835,-0.098461,0.028492,-0.035461,-0.019045,-0.041769,-0.018278,0.049495,-0.005421
4,abc.es-17477e7fd1f064e263633dba8bb2eb39d538cbf2,1,-0.028778,0.042515,0.031569,0.041072,-0.059480,0.008428,0.071166,-0.092485,...,-0.055534,0.013167,-0.081876,0.061822,-0.025959,0.020732,-0.036089,-0.035680,0.011537,-0.004418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53348,yahoo.com-ff86305c30e24585534e1064dd52483aed71...,0,0.041836,0.034119,-0.029907,-0.084251,-0.030286,0.035919,-0.020428,-0.095990,...,-0.011285,0.047606,-0.128214,0.102539,0.003723,0.010059,-0.070508,-0.070190,0.011304,0.046594
53349,yahoo.com-ffb99a82ff176686aa6cd70b5de3134418c0...,0,-0.029369,0.025984,0.004395,0.028553,-0.110529,-0.052934,-0.006775,-0.096824,...,0.017234,-0.060626,-0.095481,-0.011397,-0.051758,-0.061146,0.081326,-0.115656,-0.085305,0.033836
53350,yahoo.com-ffcc76f8ebce23fee39d5df0965ec86924b6...,0,0.095810,0.056458,0.046638,0.068726,-0.048553,-0.021271,0.007395,-0.130402,...,-0.039978,0.028349,-0.049713,-0.004752,0.070587,0.080563,-0.049167,-0.061920,0.077354,-0.077606
53351,yahoo.com-ffce6e50376bb174010c8b9c878826d13735...,0,-0.018860,0.051025,-0.066980,-0.030727,-0.037231,-0.177595,-0.013742,0.033779,...,0.012521,0.023455,-0.110944,-0.009347,0.094432,0.031634,0.103768,-0.120919,0.057426,0.046701


In [ ]:
pca = PCA(n_components=100, random_state=10)
finaldf[list(range(300, 400))] = pca.fit_transform(finaldf.drop(columns=['id','label']))
finaldf_pca=finaldf

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [ ]:
X_train, X_test = train_test_split(finaldf_pca.drop(columns='id'), test_size = 0.2, random_state = 10)
X = X_train.drop(columns= 'label')
y = X_train['label']
y_test = X_test['label']

In [ ]:
svc= SGDClassifier(max_iter=1000, tol=1e-3, class_weight= 'balanced', random_state= 100,loss="log")
model =make_pipeline(StandardScaler(),AdaBoostClassifier(n_estimators=30, base_estimator=svc,learning_rate=1))
model.fit(X,y)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('adaboostclassifier',
                 AdaBoostClassifier(base_estimator=SGDClassifier(class_weight='balanced',
                                                                 loss='log',
                                                                 random_state=100),
                                    learning_rate=1, n_estimators=30))])

In [ ]:
z = model.predict(X_test.drop(columns=["label"]).values)
model.score(X_test.drop(columns=["label"]).values, X_test['label'])
helper.output_metrics(X_test['label'],z)

Precision: 0.170411
Recall: 0.876494
F1 score: 0.285344


# Model Training

In [ ]:
finaldf = pd.read_csv('/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit/new_train_w_2_vec.csv')
finaldf

,id,label,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,abc.es-0f16d41b22631657500284f423cab35af5767edf,1,0.027974,-0.004130,0.027604,0.047051,-0.042813,-0.030466,0.031732,-0.109924,...,-0.027727,0.020912,-0.063067,0.010870,-0.007165,-0.008956,-0.055347,-0.076288,-0.034178,0.011439
1,abc.es-0fe9db0c206d6f613fb270a3557d4cfd2732b05b,0,0.036917,0.029811,-0.023043,0.000561,-0.046248,-0.024618,0.033148,-0.089768,...,-0.086267,0.005678,-0.071661,0.043474,-0.004877,-0.036751,-0.038906,-0.008480,0.027132,0.020889
2,abc.es-10d56ae3e9d99c905ede9ccf4c4b09f7c353acc9,1,-0.013969,0.008222,0.045270,-0.001446,-0.036408,-0.027278,0.060472,-0.086428,...,-0.045860,-0.022844,-0.085090,0.039385,-0.009400,0.015813,-0.040487,-0.056268,0.014432,-0.014655
3,abc.es-14c8a574d028dd3e7091cb270867993e19f56095,1,0.011252,0.018645,0.041726,0.042764,-0.007114,-0.032907,0.042889,-0.108137,...,-0.057986,-0.039835,-0.098461,0.028492,-0.035461,-0.019045,-0.041769,-0.018278,0.049495,-0.005421
4,abc.es-17477e7fd1f064e263633dba8bb2eb39d538cbf2,1,-0.028778,0.042515,0.031569,0.041072,-0.059480,0.008428,0.071166,-0.092485,...,-0.055534,0.013167,-0.081876,0.061822,-0.025959,0.020732,-0.036089,-0.035680,0.011537,-0.004418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53348,yahoo.com-ff86305c30e24585534e1064dd52483aed71...,0,0.041836,0.034119,-0.029907,-0.084251,-0.030286,0.035919,-0.020428,-0.095990,...,-0.011285,0.047606,-0.128214,0.102539,0.003723,0.010059,-0.070508,-0.070190,0.011304,0.046594
53349,yahoo.com-ffb99a82ff176686aa6cd70b5de3134418c0...,0,-0.029369,0.025984,0.004395,0.028553,-0.110529,-0.052934,-0.006775,-0.096824,...,0.017234,-0.060626,-0.095481,-0.011397,-0.051758,-0.061146,0.081326,-0.115656,-0.085305,0.033836
53350,yahoo.com-ffcc76f8ebce23fee39d5df0965ec86924b6...,0,0.095810,0.056458,0.046638,0.068726,-0.048553,-0.021271,0.007395,-0.130402,...,-0.039978,0.028349,-0.049713,-0.004752,0.070587,0.080563,-0.049167,-0.061920,0.077354,-0.077606
53351,yahoo.com-ffce6e50376bb174010c8b9c878826d13735...,0,-0.018860,0.051025,-0.066980,-0.030727,-0.037231,-0.177595,-0.013742,0.033779,...,0.012521,0.023455,-0.110944,-0.009347,0.094432,0.031634,0.103768,-0.120919,0.057426,0.046701


In [ ]:
pca = PCA(n_components=100, random_state=10)
finaldf[list(range(300, 400))] = pca.fit_transform(finaldf.drop(columns=['id','label']))
finaldf_pca=finaldf

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [ ]:
X = finaldf_pca.drop(columns=['id',"label"])
y =  finaldf_pca['label']

In [ ]:
svc= SGDClassifier(max_iter=1000, tol=1e-3, class_weight= 'balanced', random_state= 100,loss="log")
model =make_pipeline(StandardScaler(),AdaBoostClassifier(n_estimators=30, base_estimator=svc,learning_rate=1))
model.fit(X,y)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('adaboostclassifier',
                 AdaBoostClassifier(base_estimator=SGDClassifier(class_weight='balanced',
                                                                 loss='log',
                                                                 random_state=100),
                                    learning_rate=1, n_estimators=30))])

In [ ]:

Pkl_Filename = "/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit/New_SGDClassifier.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

In [ ]:

Pkl_Filename = "/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit/new_pca.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(pca, file)